In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
df = pd.read_csv('/content/drive/My Drive/coursepaper2020/has_subject.csv')

In [0]:
df.head()

,DOI,title,abstract,subject
NaN,DOI,title,abstract,subject
1468.0,10.1163/1568525043083505,['Aristotle FR. 44 Rose: Midas and Silenus'],<jats:sec><jats:title>Abstract</jats:title><ja...,"['Classics', 'Linguistics and Language', 'Arch..."
1469.0,10.1163/1568525043083532,"[""Loca Loquuntur. Lucretius' Explanation of th...",<jats:sec><jats:title>Abstract</jats:title><ja...,"['Classics', 'Linguistics and Language', 'Arch..."
1470.0,10.1163/1568525043083541,['Poverty and Demography: The Case of the Grac...,<jats:sec><jats:title>Abstract</jats:title><ja...,"['Classics', 'Linguistics and Language', 'Arch..."
1471.0,10.1163/1568525043083514,['Old Persian in Athens Revisited (Ar. Ach. 10...,<jats:sec><jats:title>Abstract</jats:title><ja...,"['Classics', 'Linguistics and Language', 'Arch..."


In [0]:
df.shape

(2345239, 4)

In [0]:
df = df.iloc[1:]

In [0]:
df.drop_duplicates(keep=False, ignore_index=True, inplace=True)

In [0]:
df.head()

,DOI,title,abstract,subject
0,10.1163/1568525043083505,['Aristotle FR. 44 Rose: Midas and Silenus'],<jats:sec><jats:title>Abstract</jats:title><ja...,"['Classics', 'Linguistics and Language', 'Arch..."
1,10.1163/1568525043083532,"[""Loca Loquuntur. Lucretius' Explanation of th...",<jats:sec><jats:title>Abstract</jats:title><ja...,"['Classics', 'Linguistics and Language', 'Arch..."
2,10.1163/1568525043083541,['Poverty and Demography: The Case of the Grac...,<jats:sec><jats:title>Abstract</jats:title><ja...,"['Classics', 'Linguistics and Language', 'Arch..."
3,10.1163/1568525043083514,['Old Persian in Athens Revisited (Ar. Ach. 10...,<jats:sec><jats:title>Abstract</jats:title><ja...,"['Classics', 'Linguistics and Language', 'Arch..."
4,10.1163/1568527053083412,['Religion and Violence: What can Sociology Of...,<jats:sec><jats:title>Abstract</jats:title><ja...,"['Religious studies', 'History']"


In [0]:
df.to_csv('/content/drive/My Drive/coursepaper2020/has_subject_prettier.csv')

In [0]:
subj2field = pd.read_csv('subj2field.csv')

In [0]:
subj2field.head()

,Subject,Field,Hyperfield
0,Accounting,Business,Business
1,Acoustics and Ultrasonics,Physics,Formal
2,Advanced and Specialised Nursing,Medicine,Natural
3,Aerospace Engineering,Physics,Formal
4,Ageing,Biology,Natural


In [0]:
subj2field = subj2field.to_dict('records')

In [0]:
subj2field = {d['Subject']: d['Field'] for d in subj2field}

In [0]:
from collections import Counter

def get_field(subj_list):
  global subj2field
  fields = []
  for subj in subj_list:
    try:
      fields.append(subj2field[subj.strip()])
    except KeyError:
      try:
        fields.append(subj2field[subj.split(',')[0]])
      except KeyError:
        try:
          fields.append(subj2field[subj.replace(', ', ' ')])
        except KeyError:
          print(f'{subj} not in subj2field')
  most_common = Counter(fields).most_common()[0][0]
  return most_common

In [0]:
get_field(['Accounting', 'Acoustics and Ultrasonics', 'Aerospace Engineering'])

'Physics'

In [0]:
from tqdm import tqdm_notebook

with open('fields.txt', 'w', encoding='utf-8') as f:
  for _, row in tqdm_notebook(df.iterrows()):
    if not isinstance(row['subject'], list):
      row['subject'] = eval(row['subject'])
    print(row['DOI'], get_field(row['subject']),
          sep=',', file=f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [0]:
fields = pd.read_csv('/content/drive/My Drive/coursepaper2020/fields.csv', names=['DOI', 'field'])

In [0]:
fields.head()

,DOI,field
0,10.1163/1568525043083505,Humanities
1,10.1163/1568525043083532,Humanities
2,10.1163/1568525043083541,Humanities
3,10.1163/1568525043083514,Humanities
4,10.1163/1568527053083412,Humanities


In [0]:
df = pd.merge(df, fields, on='DOI')

In [0]:
df.head()

,DOI,title,abstract,subject,field
0,10.1163/1568525043083505,['Aristotle FR. 44 Rose: Midas and Silenus'],<jats:sec><jats:title>Abstract</jats:title><ja...,"['Classics', 'Linguistics and Language', 'Arch...",Humanities
1,10.1163/1568525043083532,"[""Loca Loquuntur. Lucretius' Explanation of th...",<jats:sec><jats:title>Abstract</jats:title><ja...,"['Classics', 'Linguistics and Language', 'Arch...",Humanities
2,10.1163/1568525043083541,['Poverty and Demography: The Case of the Grac...,<jats:sec><jats:title>Abstract</jats:title><ja...,"['Classics', 'Linguistics and Language', 'Arch...",Humanities
3,10.1163/1568525043083514,['Old Persian in Athens Revisited (Ar. Ach. 10...,<jats:sec><jats:title>Abstract</jats:title><ja...,"['Classics', 'Linguistics and Language', 'Arch...",Humanities
4,10.1163/1568527053083412,['Religion and Violence: What can Sociology Of...,<jats:sec><jats:title>Abstract</jats:title><ja...,"['Religious studies', 'History']",Humanities


In [0]:
import re
 
def preprocess_text(text):
  prepr = text.lower()
  prepr = re.sub(r'<.+?>', ' ', prepr)
  prepr = re.sub(r'[^a-z0-9]', ' ', prepr)
  prepr = re.sub(r'[\s\r\n]{2,}', ' ', prepr).strip()
  return prepr

In [0]:
df.abstract[0]

"<jats:sec><jats:title>Abstract</jats:title><jats:p>Scholars have identified two supposedly separate folk-tale patterns: (a) the questing hero's initial encounter with an ambivalent helper figure; (b) the capture of a demonic figure from whom crucial information is extracted. This article establishes that (a) and (b) are identical aspects of one and the same pattern and that the story of Midas' encounter with Silenus exemplifies this pattern. Two Appendixes show how Silenus resembles that primeval and widely disseminated figure the Wild Man; and that the folk-tale sequence whereby one helper sends on the hero to another helper (or helpers) lurks beneath the surface of several stories that exemplify the relevant pattern.</jats:p>\n         </jats:sec>"

In [0]:
preprocess_text(df.abstract[0])

'abstract scholars have identified two supposedly separate folk tale patterns a the questing hero s initial encounter with an ambivalent helper figure b the capture of a demonic figure from whom crucial information is extracted this article establishes that a and b are identical aspects of one and the same pattern and that the story of midas encounter with silenus exemplifies this pattern two appendixes show how silenus resembles that primeval and widely disseminated figure the wild man and that the folk tale sequence whereby one helper sends on the hero to another helper or helpers lurks beneath the surface of several stories that exemplify the relevant pattern'

In [0]:
df.shape

(2307763, 5)

In [0]:
from tqdm import tqdm_notebook
 
with open('/content/drive/My Drive/coursepaper2020/prepr_texts.csv', 'w', encoding='utf-8') as f:
  print('DOI,prepr_title,prepr_abstract', file=f)
 
  for _, row in tqdm_notebook(df.iterrows(), total=df.shape[0]):
    if not isinstance(row['title'], list):
      row['title'] = eval(row['title'])
    
    if isinstance(row['title'], list):
      row['title'] = row['title'][0]
 
    print(row['DOI'], preprocess_text(row['title']), preprocess_text(row['abstract']),
          sep=',', file=f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [0]:
row['abstract']

'<jats:sec>\n<jats:title>:</jats:title>\n<jats:p>Drug nanocrystals offer an attractive approach for improving the solubility and dissolution rate of poorly soluble drugs which\naccounts for nearly 40 % newly discovered drug molecules. Both methods for manufacturing drug nanocrystals have high industrial\nacceptability for being simple and easy to scale which is evident from the number of approved products available in the market. Ability to\nmodify multiple aspects of dosage form like bioavailability, release pattern and dosage form requirement along with flexibility in choosing final\ndosage form starting from the tablet, capsule, suspension to parenteral one, have made nanocrystal technology one of the very promising and\nadaptable technology for dosage form design.</jats:p>\n</jats:sec>'